In [1]:
# %% Import Libraries and Data

# Third-party imports
import numpy as np
import pyansys

# Local imports
from util_loads import Propeller, Airfoil, Loadcase
from util_mapdl import Material
from femodel import Threepartmodel

# %% Instantiate Airfoils and assign radial sections

airfoil = Airfoil('mf3218.xfo', 500000, iter_limit=600)
airfoildick = Airfoil('mf3218-dick.xfo', 500000, iter_limit=600)
rectangle = Airfoil('rectangle2.txt',500000)

# %% Instantiate Propeller and assign geometry and airfoils

propeller = Propeller(number_of_blades=2,
                      tip_radius=0.412,
                      hub_radius=0.04,
                      )

propeller.geometry = np.array([[0.10,0.078,0],
                               [0.121, 0.078, 0.],
                               [0.155, 0.100, 5.99],
                               [0.223, 0.160, 17.97],
                               [0.345, 0.149, 14.44],
                               [0.417, 0.142, 12.68],
                               [0.490, 0.135, 11.18],
                               [0.563, 0.128, 9.94],
                               [0.636, 0.121, 8.97],
                               [0.709, 0.114, 8.26],
                               [0.782, 0.107, 7.81],
                               [0.854, 0.100, 7.63],
                               [0.947, 0.091, 7.5],
                               [1., 0.066, 7.5],                               
                               ])

propeller.sections = [[0.121, airfoil],
                      [0.223, airfoil],
                      [1., airfoil]]

propeller.geometric_sections = [[0.121, rectangle],
                                [0.223, airfoildick],
                                [1., airfoildick]]

for x in propeller.sections:
    x[1].set_polar(alpha_start=-7, alpha_stop=20, alpha_inc=0.25) 
    
airfoil.xrotor_characteristics['Cm'] = -0.14
airfoil.xrotor_characteristics['d(Cl)/d(alpha)'] = 6.28
airfoil.xrotor_characteristics['Minimum Cl'] = 0.

# %% Instantiate Loadcases

propeller.add_loadcase(loadcase=Loadcase(name='Max RPM', flight_speed=0.01))

propeller.loadcases[0][0].set_data('rpm',4000)

# %% Calculate loads

propeller.calc_loads()
propeller.set_load_envelope()

In [3]:
X, Y, Cp_suc, Cp_pres = propeller.pressure_distribution('envelope')

Unconverged point: Airfoil mf3218.xfo cl 0.872. Filling with 0.
Unconverged point: Airfoil mf3218.xfo cl 0.84. Filling with 0.
Unconverged point: Airfoil mf3218.xfo cl 0.84. Filling with 0.
Unconverged point: Airfoil mf3218.xfo cl 0.795. Filling with 0.
Unconverged point: Airfoil mf3218.xfo cl 0.795. Filling with 0.
Unconverged point: Airfoil mf3218.xfo cl 0.795. Filling with 0.
Unconverged point: Airfoil mf3218.xfo cl 0.795. Filling with 0.
Unconverged point: Airfoil mf3218.xfo cl 0.795. Filling with 0.
Unconverged point: Airfoil mf3218.xfo cl 0.795. Filling with 0.


In [17]:
import matplotlib.pyplot as plt
%matplotlib widget

In [41]:
fig = plt.figure(figsize=(15,15))
ax = plt.axes(projection="3d")

ax.plot_surface(Y, X, -Cp_suc,cmap='plasma')
ax.plot_surface(Y, X, -Cp_pres,cmap='plasma')


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [38]:
with open('pdist.txt', 'w') as f:
    for idline, line in enumerate(X):
        for idrow, row in enumerate(line):
            f.write('(%s, %s, %s)\n' % (Y[idline, idrow], X[idline, idrow], -Cp_suc[idline, idrow]))
            if idrow == 29:
                f.write('\n')

In [43]:
propeller.loadcases[0][1]['oper'].plot(x='r/R', y='CL')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:xlabel='r/R'>